# Prototype Model
---

* Regression
* 2009 data로 class1,2,3 10fold


https://docs.google.com/presentation/d/1cI3teBcQoGBhfrdVrv9rnM4jHGaPToDS7Zh7t0q7GKg/edit#slide=id.g97e66187c6_0_6

In [1]:
import tensorflow as tf

In [2]:
from random import seed as random_seed
random_seed(50)
from numpy.random import seed as np_random_seed
np_random_seed(50)

tf.random.set_seed(50)

In [3]:
import os

os.sys.path.append(r'/docker_mnt/data5/jin/jin/python/')

os.sys.path

['/whbom/Malocclusion/bom_regression/model/inceptionresnet',
 '/root/miniconda3/lib/python37.zip',
 '/root/miniconda3/lib/python3.7',
 '/root/miniconda3/lib/python3.7/lib-dynload',
 '',
 '/root/miniconda3/lib/python3.7/site-packages',
 '/root/miniconda3/lib/python3.7/site-packages/locket-0.2.1-py3.7.egg',
 '/root/miniconda3/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython',
 '/docker_mnt/data5/jin/jin/python/']

In [4]:
import tensorflow as tf
tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2978313750507894698]

In [5]:
%env PYTHONHASHSEED=0
from random import seed as random_seed
random_seed(42)
from numpy.random import seed as np_random_seed
np_random_seed(42)

import glob
import os
import pprint
from tqdm import tqdm

from IPython import display
import ipywidgets as widgets

# Data Science
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('seaborn-white')
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')

import cv2 as cv

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import models, layers, activations, initializers, regularizers, optimizers, losses, callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

env: PYTHONHASHSEED=0


## Paths
---


In [18]:
so_type = 'so_rl'

input_path_root = r'../../../data/reprocess_data/fullRange_D'

loss_param=0.5
MODEL_TYPE="prototype"
version="v1"

# <model 저장>
# output_path_root = r'../results/'
output_path_root = f'../../results/multiout/{MODEL_TYPE}/{loss_param}_full_range_1951_multiO2/{version}'
output_path = os.path.join(output_path_root,  so_type)
output_path_weight = os.path.join(output_path, r'weights/')

try:
    os.makedirs(output_path_weight) 
    
except FileExistsError as err:      
    print(err)
# else:
#     print(output_path)
#     print(output_path_weight)

In [19]:
input_path_root

'../../../data/reprocess_data/fullRange_D'

In [20]:
output_path_root

'../../results/multiout/prototype/0.5_full_range_1951_multiO2/v1'

## Data
---
- Class 0 : 분별하기 어려운 data (65 sample) -> 제외

- Class 3 : 하악이 상악보다 앞으로 돌출된 경우 (1136 sample) -> -1.0
- Class 1 : 상하악의 맞물림 상태는 정상이지만 치열이 고르지 않은 경우 (1707 sample) -> 0.0
- Class 2 : 상악이 하악보다 앞으로 돌출된 경우 (1175 sample) -> 1.0


In [21]:
def convert_label_class2reg(label_df):
    label_r = np.copy(label_df.angle_class_r)
    label_df.loc[label_r == 1, 'angle_class_r'] = 0
    label_df.loc[label_r == 2, 'angle_class_r'] = 1
    label_df.loc[label_r == 3, 'angle_class_r'] = -1
    
    label_l = np.copy(label_df.angle_class_l)
    label_df.loc[label_l == 1, 'angle_class_l'] = 0
    label_df.loc[label_l == 2, 'angle_class_l'] = 1
    label_df.loc[label_l == 3, 'angle_class_l'] = -1
    
    return label_df

def generate_data_by_patient(x, y): #y_distance):
    if x.shape[-1] == 12:
        x = np.concatenate([x[..., :3], x[..., 3:6], x[..., 6:9], x[..., 9:]]) 
        y = np.concatenate([y[:, 0], y[:, 1], y[:, 2], y[:, 3]])
    elif x.shape[-1] == 6:
        x = np.concatenate([x[..., :3], x[..., 3:]]) # Right side,Left side !!!
        
        y_r_test= to_categorical(y[:,0], num_classes=3)
        y_l_test= to_categorical(y[:,1], num_classes=3)
        
        y = np.concatenate([y[:, 0], y[:, 1]])
        y_onehot=np.concatenate((y_r_test,y_l_test),axis=0)
        
#         y_distance = np.concatenate([y_distance[:, 0], y_distance[:, 1]])
    
    return x, y, y_onehot #, y_distance

# def generate_info_df_by_patient(info_df):
#     info_df_r = info_df.copy()
#     for row_idx, info in enumerate(info_df_r['info']):
#         info_df_r.iloc[row_idx] = info + '_r'
        
#     info_df_l = info_df.copy()
#     for row_idx, info in enumerate(info_df_l['info']):
#         info_df_l.iloc[row_idx] = info + '_l'
    
#     info_df = pd.concat((info_df_r, info_df_l), ignore_index=True)
#     del info_df_r, info_df_l
    
#     info_df1 = info_df.copy()
        
#     info_df2 = info_df.copy()
#     for row_idx, info in enumerate(info_df2['info']):
#         info_df2.iloc[row_idx] = info.replace('01A1', '02B1')
    
#     info_df = pd.concat((info_df1, info_df2), ignore_index=True)
#     del info_df1, info_df2

#     return info_df

In [22]:
x = np.load(os.path.join(input_path_root, f'fullRange_D.npy'))
label_df = pd.read_csv(os.path.join(input_path_root, f'fullRange_D.csv'))
label_df = convert_label_class2reg(label_df)

y = label_df.loc[:, ['angle_class_r', 'angle_class_l']].to_numpy()
y_distance= label_df.loc[:, ['distance(r)', 'distance(l)']].to_numpy()

y_distance = y_distance.astype(np.float32)

print(x.max())
x=preprocess_input(x)
print(x.max())


255


NameError: name 'preprocess_input' is not defined

In [ ]:
#---------onehot function test------------------
x_tt, y_tt, y_onehot_tt=generate_data_by_patient(x,y)

y_onehot_tt_r=y_onehot_tt[:(y_onehot_tt.shape[0]//2)]
y_onehot_tt_l=y_onehot_tt[(y_onehot_tt.shape[0]//2):]

y_onehot_tt_r=list(y_onehot_tt_r)
y_onehot_tt_l=list(y_onehot_tt_l)


label_df["one-hot-r"]=y_onehot_tt_r
label_df["one-hot-l"]=y_onehot_tt_l
label_df
#---------------------------------------------

In [ ]:
print(x.dtype, x.shape)
print(y.dtype, y.shape)
print()
display.display(label_df.info())
display.display(label_df)

plt.style.use('seaborn-whitegrid')
plt.figure()

plt.title('Label')
sns.countplot(np.concatenate([label_df.angle_class_r, label_df.angle_class_l]))

plt.show()
plt.style.use('seaborn-white')

In [ ]:
cnt = np.concatenate([label_df.angle_class_r, label_df.angle_class_l])


print("calss -1:",len(cnt[cnt==-1]))
print("calss 0:",len(cnt[cnt==0]))
print("calss 1:",len(cnt[cnt==1]))

## Model
---

In [ ]:
def build_model():
    base_model = InceptionResNetV2(include_top=False, weights=None, input_shape=(None, None, 3))

    x = base_model.output

    x = layers.GlobalAveragePooling2D(name='gap')(x)
    x = layers.Dense(256, activation='relu', name='dense')(x)
    x = layers.Dropout(0.2, name='dropout')(x)
    
    one_hot_prediction=layers.Dense(3, activation='softmax', name='classification')(x)

    
    model = models.Model(inputs=base_model.input, outputs=[regression_prediction,one_hot_prediction])
#     model = models.Model(inputs=base_model.input, outputs=[regression_prediction,one_hot_prediction,distance_prediction])
    
    return model

In [ ]:
model=build_model()

In [ ]:
model.summary()

## Train the Model
---

In [18]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
fine_tun = False

for kfold_idx, (train_idxs, test_idxs) in enumerate(kfold.split(x, y[:, 0])): #left???

    if kfold_idx ==4:
        display.clear_output(wait=True)
        print(f'K-Fold CV: {kfold_idx + 1:02d}/{kfold.n_splits:02d}')

        # 1. Data
        # ----------------------------------------

    #------DEBUG--------:
    #     print(test_idxs)     
    #     print(np.unique(y[test_idxs, 0], return_counts=True))
    #-------------------   

        x_train = x[train_idxs]
        y_train = y[train_idxs]
        y_distance_train =y_distance[train_idxs]
    #     label_df_train = label_df.iloc[train_idxs]

        x_train, x_val, y_train, y_val, y_distance_train, y_distance_val = train_test_split(x_train, y_train, y_distance_train, test_size=0.2, 
                                                                                        random_state=42, stratify=y_train[:, 0])

        x_test = x[test_idxs]
        y_test = y[test_idxs]
        y_distance_test =y_distance[test_idxs]
    #     label_df_test = label_df.iloc[test_idxs]

        #distance 누락!
        x_train, y_train,y_train_onehot, y_train_distance = generate_data_by_patient(x_train, y_train, y_distance_train)
        x_val, y_val, y_val_onehot, y_val_distance = generate_data_by_patient(x_val, y_val, y_distance_val)
        x_test, y_test, y_test_onehot, y_test_distance = generate_data_by_patient(x_test, y_test, y_distance_test)

        weight_for_m1 = (1 / len(y_train[y_train==-1]))*(len(y_train))/3.0 
        weight_for_0 = (1 / len(y_train[y_train==0]))*(len(y_train))/3.0 
        weight_for_1 = (1 / len(y_train[y_train==1]))*(len(y_train))/3.0

        class_weight2=[weight_for_0,weight_for_1,weight_for_m1]
        

        # 2. Paths
        # ----------------------------------------

        output_path_kfold = os.path.join(output_path, f'kfold_{kfold_idx + 1:02d}/')
        output_path_weight = os.path.join(output_path_kfold, r'weights/')

        if os.path.isdir(output_path_weight)==False:
            os.makedirs(output_path_weight)
            
      
        
        model = build_model()
        model.save(os.path.join(output_path, r'model.hdf5'))

        
        model.compile(optimizer=optimizers.RMSprop(lr=1e-3),
              loss={'regression_prediction':"mae",'classification':"categorical_crossentropy",'distance_prediction':"mae"},
              loss_weights={'regression_prediction':1,'classification':1,'distance_prediction':loss_param},
              metrics={'regression_prediction':"mae",'classification':"acc",'distance_prediction':"mae"})

        print(len(model.trainable_weights))

        # 4. Train the Model.
        # ----------------------------------------

        # Callbacks
        checkpointer = callbacks.ModelCheckpoint(os.path.join(output_path_weight, 'weights_{epoch:08d}_{val_loss:.4g}.hdf5'),
                                                 save_weights_only=True)
        csv_logger = callbacks.CSVLogger(os.path.join(output_path_kfold, r'log.csv'), append=True)
        learning_sch = callbacks.LearningRateScheduler(scheduler)

        # Training batch size 16 -> 8
        model.fit(x_train,{'regression_prediction':y_train, 'classification':y_train_onehot,'distance_prediction':y_train_distance},
                  batch_size=16,
                  epochs=50,
                  validation_data=(x_val, {'regression_prediction':y_val,'classification':y_val_onehot,'distance_prediction':y_val_distance}),
                  callbacks=[checkpointer, csv_logger],
                  initial_epoch=0,
                  verbose=1)

        K.clear_session()
        del model
    


K-Fold CV: 06/10
[  10   30   45   55   61   62   82   85  107  110  111  118  119  126
  131  137  143  144  156  160  179  188  195  202  231  232  240  246
  252  272  279  292  295  305  317  318  323  329  341  343  368  374
  379  394  396  400  403  408  409  426  428  437  447  456  488  500
  507  510  511  516  517  540  543  552  560  568  577  586  608  612
  615  629  659  663  670  686  693  739  742  798  808  815  824  829
  830  831  841  845  849  852  859  877  915  918  966  968  970  978
  990 1005 1038 1044 1045 1062 1071 1072 1088 1105 1113 1115 1120 1130
 1137 1139 1141 1146 1151 1154 1171 1186 1195 1200 1205 1210 1243 1259
 1263 1266 1269 1280 1288 1292 1297 1301 1306 1350 1356 1358 1369 1398
 1402 1409 1432 1436 1447 1473 1479 1480 1514 1515 1534 1536 1539 1550
 1570 1616 1617 1646 1654 1657 1675 1682 1695 1701 1708 1721 1725 1728
 1729 1733 1738 1758 1759 1765 1769 1787 1801 1803 1812 1830 1839 1842
 1843 1844 1860 1866 1869 1885 1888 1921 1924 1942 1945 1948

KeyError: 'val_loss'

## Evaluation
---

In [ ]:
info_df = label_df.copy()
info_df

In [ ]:
info_df.iloc[[0,3,5,6,7]]

In [ ]:
output_path

In [ ]:
metrics = []
predictions_kfold = []
labels_kfold = []
distance_regression = [] 

total_info =[]
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# model_parm_name="D"+depth+"_"+dropout_rat

for kfold_idx, (train_idxs, test_idxs) in enumerate(kfold.split(x, y[:, 0])):
    print(f'K-Fold CV: {kfold_idx + 1:02d}/{kfold.n_splits:02d}')
    
    # 1. Data
    # ----------------------------------------
    
    if kfold_idx==0:

        x_test = x[test_idxs]
        y_test = y[test_idxs]

        label_df_test = label_df.iloc[test_idxs]

        x_test, y_test, y_test_onehot = generate_data_by_patient(x_test, y_test)

        # 2. Paths
        # ----------------------------------------

    #     output_path_kfold = os.path.join(output_path, model_parm_name, f'kfold_{kfold_idx + 1:02d}/')
        output_path_kfold = os.path.join(output_path,  f'kfold_{kfold_idx + 1:02d}/')
        output_path_weight = os.path.join(output_path_kfold, r'weights/')

        # 3. Plot Learning Curves.
        # ----------------------------------------

        # log 저장: loss, mae, val_loss, val_mae
        log_df = pd.read_csv(os.path.join(output_path_kfold, r'log.csv'))


        plt.style.use('seaborn-whitegrid')
        plt.figure(figsize=(25, 6))

        plt.subplot(1,3,1)
        plt.title(f'Learning Curves ({kfold_idx + 1:02d}/{kfold.n_splits:02d})')
        plt.plot(log_df.loss)
        plt.plot(log_df.val_loss)
        plt.legend(('Train Loss', 'Validation Loss'))

        plt.subplot(1,3,2)
        plt.plot(log_df.classification_acc)
        plt.plot(log_df.val_classification_acc)
        plt.ylim(0, 1.5)
        plt.legend(('Train Classification MAE', 'Validation Classification MAE'))

#         plt.subplot(2,2,3)
#         plt.plot(log_df.distance_prediction_mae)
#         plt.plot(log_df.val_distance_prediction_mae)
#         plt.ylim(0, 1.5)
#         plt.legend(('Train distance Classification MAE', 'Validation distance Classification MAE'))

        plt.subplot(1,3,3)
        plt.plot(log_df.regression_prediction_mae)
        plt.plot(log_df.val_regression_prediction_mae)
        plt.ylim(0, 1.5)
        plt.legend(('Train regression Classification MAE', 'Validation regression Classification MAE'))

        plt.show()
        plt.style.use('seaborn-white')
        print(f'Min. validation loss epoch: {log_df.val_loss.idxmin() + 1}')
        print(f'Max. Classification MAE epoch: {log_df.val_classification_acc.idxmax() + 1}')
#         print(f'Max. distance validation MAE epoch: {log_df.val_distance_prediction_mae.idxmax() + 1}')
        print(f'Max. regression validation MAE epoch: {log_df.val_regression_prediction_mae.idxmax() + 1}')



        info_df = label_df.copy()

        result_info_df = info_df.iloc[test_idxs]

        inference_df = result_info_df.copy()

        # 4. Load a Model.
        # ----------------------------------------

        K.clear_session()

        #sgcwhb/Malocclusion/results/prototype_regression_so_img/so_rl/model.hdf5
        model = models.load_model(os.path.join(output_path, r'model.hdf5'))
        #validation mae가 가장 적은 model weight load
        model.load_weights(glob.glob(os.path.join(output_path_weight, f'weights_{log_df.val_loss.idxmin() + 1:08d}*.hdf5'))[0])

        # 5. Display
        # ----------------------------------------


        prediction = model.predict(x_test)

        prediction_distance_r = prediction[0][:int(prediction[0].shape[0]/2),0] #Right
        prediction_distance_l = prediction[0][int(prediction[0].shape[0]/2):,0] #left

        r = prediction_distance_r.copy()
        l = prediction_distance_l.copy()

        #-1~1사이 score
        r = np.round(r) #반올림
        r[r<-1]= -1
        r[r>1] = 1


        l = np.round(l)
        l[l<-1]= -1
        l[l>1] = 1    

    #     print(l.shape)
    #     print(r.shape)
    #     print(info_df.shape)
    #     print(result_info_df.shape)

        inference_df['prediction_class_r'] = r
        inference_df['prediction_class_l'] =l
        inference_df['regression_distance_r'] = np.round(prediction_distance_r,3)
        inference_df['regression_distance_l'] = np.round(prediction_distance_l,3)




        inference_df.to_csv(os.path.join(output_path_kfold, r'info.csv'))
        total_info.append(inference_df)

    total_df = pd.concat(total_info)
    total_df.to_csv(os.path.join(output_path,'total.csv'))


In [ ]:
metrics = []
predictions_kfold = []
labels_kfold = []
distance_regression = [] 

total_info =[]
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# model_parm_name="D"+depth+"_"+dropout_rat

for kfold_idx, (train_idxs, test_idxs) in enumerate(kfold.split(x, y[:, 0])):
    
    log_df = pd.read_csv(os.path.join(output_path_kfold, r'log.csv'))
    
    
    plt.style.use('seaborn-whitegrid')
    plt.figure(figsize=(16, 6))
    
    plt.subplot(2,2,1)
    plt.title(f'Learning Curves ({kfold_idx + 1:02d}/{kfold.n_splits:02d})')
    plt.plot(log_df.loss)
    plt.plot(log_df.val_loss)
    plt.legend(('Train Loss', 'Validation Loss'))
    
    plt.subplot(2,2,2)
    plt.plot(log_df.classification_acc)
    plt.plot(log_df.val_classification_acc)
    plt.ylim(0, 1.5)
    plt.legend(('Train Classification acc', 'Validation Classification acc'))
    
    plt.subplot(2,2,3)
    plt.plot(log_df.distance_prediction_acc)
    plt.plot(log_df.val_distance_prediction_acc)
    plt.ylim(0, 1.5)
    plt.legend(('Train distance Classification acc', 'Validation distance Classification acc'))
    
    plt.subplot(2,2,4)
    plt.plot(log_df.regression_prediction_acc)
    plt.plot(log_df.val_regression_prediction_acc)
    plt.ylim(0, 1.5)
    plt.legend(('Train regression Classification acc', 'Validation regression Classification acc'))

    plt.show()
    plt.style.use('seaborn-white')
    print(f'Min. validation loss epoch: {log_df.val_loss.idxmin() + 1}')
    print(f'Min. Classification MAE epoch: {log_df.val_classification_acc.idxmin() + 1}')
    print(f'Min. distance validation MAE epoch: {log_df.val_distance_prediction_acc.idxmin() + 1}')
    print(f'Min. regression validation MAE epoch: {log_df.val_regression_prediction_acc.idxmin() + 1}')
    
    
    
    info_df = label_df.copy()
    
    result_info_df = info_df.iloc[test_idxs]
    
    inference_df = result_info_df.copy()
    
    # 4. Load a Model.
    # ----------------------------------------
    
    K.clear_session()
    
    


In [ ]:
log_df.val_classification_acc.max()

In [ ]:
total_df

In [ ]:
kfold_results_df = pd.DataFrame(metrics, columns=('test_loss', 'test_mae'))


In [ ]:
data = total_df['angle_class_r'].append(total_df['angle_class_l'])
prediction =  total_df['prediction_class_r'].append(total_df['prediction_class_l'])
data = np.array(data)
prediction=np.array(prediction,dtype = int)

cm = confusion_matrix(data, prediction, labels=[-1, 0, 1])

print(cm)
print("")
print(f'Accuracy: {(cm[0, 0] + cm[1, 1] + cm[2, 2]) / cm.sum():.3f}')

In [ ]:
kfold_results_df.describe()

In [ ]:
np.sum(cm)